In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [3]:
df = pd.read_csv("../data/yeast.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'yeast'

In [4]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [33]:
def fit_trees():
    learning_rate=0.1
    max_depth=2
    bins=8
    lam=10

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 5
    verbose = 1
    tolerance=0.05

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 5
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

with open('./benchmark/'+dataset+'.csv','w') as f:
        f.writelines("")

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

12 78 1
24 142 2
29 153 3
35 201 4
42 238 5
47 247 6
49 265 7
49 237 8
51 247 9
53 255 10
57 289 11
58 278 12
59 289 13
63 323 14
68 352 15
70 362 16
72 372 17
73 373 18
74 382 19
76 394 20
76 394 21
78 402 22
79 417 23
79 415 24
80 424 25
82 440 26
88 480 27
88 478 28
90 496 29
90 488 30
90 478 31
91 455 32
92 456 33
94 470 34
94 470 35
95 473 36
95 473 37
97 483 38
97 481 39
98 492 40
99 499 41
101 519 42
102 522 43
104 532 44
104 518 45
106 532 46
108 544 47
108 540 48
108 540 49
109 545 50
109 543 51
110 542 52
111 541 53
112 554 54
112 554 55
113 543 56
113 541 57
114 558 58
116 576 59
116 564 60
117 559 61
118 566 62
119 571 63
119 569 64
123 597 65
124 590 66
124 588 67
124 584 68
125 593 69
126 598 70
127 603 71
128 606 72
129 613 73
129 609 74
128 602 75
128 600 76
129 617 77
130 616 78
131 633 79
131 631 80
131 631 81
132 654 82
134 678 83
134 678 84
135 691 85
137 709 86
138 714 87
140 722 88
141 727 89
141 719 90
141 719 91
141 717 92
141 717 93
141 715 94
142 722 95
142 72

c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhou

18 104 1
25 147 2
37 209 3
43 251 4
47 243 5
52 292 6
54 308 7
56 298 8
59 315 9
61 331 10
65 383 11
68 406 12
69 413 13
71 421 14
74 440 15
78 456 16
81 471 17
82 474 18
85 503 19
85 503 20
87 515 21
88 504 22
89 515 23
92 532 24
93 543 25
94 558 26
95 555 27
97 547 28
97 531 29
97 529 30
101 549 31
102 564 32
105 587 33
107 587 34
109 611 35
112 630 36
113 639 37
116 670 38
119 675 39
120 666 40
122 676 41
123 669 42
124 686 43
127 715 44
128 708 45
129 719 46
130 724 47
131 731 48
131 729 49
132 738 50
132 728 51
132 720 52
133 713 53
134 706 54
135 703 55
138 714 56
139 719 57
139 717 58
140 726 59
142 730 60
144 750 61
144 750 62
144 736 63
146 746 64
147 749 65
147 747 66
149 769 67
153 797 68
155 813 69
156 810 70
159 837 71
159 833 72
159 825 73
159 823 74
162 862 75
162 848 76
162 844 77
164 868 78
164 862 79
166 878 80
166 870 81
166 862 82
166 860 83
166 856 84
167 867 85
170 882 86
171 889 87
171 887 88
171 871 89
171 869 90
172 868 91
172 862 92
176 904 93
178 916 94
179 9

c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


17 119 1
29 169 2
38 234 3
45 281 4
57 323 5
61 333 6
63 357 7
65 363 8
66 354 9
74 442 10
78 442 11
84 490 12
87 499 13
87 495 14
91 509 15
92 508 16
92 506 17
94 514 18
96 500 19
97 507 20
100 536 21
101 535 22
103 543 23
103 543 24
107 603 25
109 605 26
110 588 27
110 574 28
111 585 29
112 598 30
112 598 31
116 626 32
117 633 33
117 629 34
118 638 35
121 653 36
121 641 37
124 658 38
124 658 39
124 656 40
124 656 41
125 669 42
125 653 43
125 651 44
126 658 45
128 674 46
130 690 47
132 688 48
132 686 49
133 697 50
135 725 51
135 725 52
136 720 53
140 750 54
141 755 55
141 741 56
142 758 57
144 772 58
144 752 59
144 744 60
144 742 61
144 738 62
144 738 63
144 738 64
145 739 65
145 733 66
145 733 67
147 751 68
147 743 69
147 743 70
150 764 71
152 782 72
153 787 73
155 797 74
155 795 75
155 791 76
156 800 77
157 807 78
157 805 79
157 803 80
158 808 81
159 821 82
159 821 83
159 821 84
159 815 85
159 813 86
160 822 87
162 842 88
162 834 89
162 824 90
162 824 91
163 829 92
164 838 93
165 83

...

19 103 1
29 171 2
30 156 3
35 185 4
40 210 5
49 245 6
51 241 7
54 260 8
62 320 9
63 327 10
64 330 11
68 344 12
68 342 13
69 345 14
70 346 15
72 358 16
74 362 17
74 358 18
75 367 19
76 374 20
76 368 21
82 432 22
84 442 23
87 465 24
90 492 25
91 499 26
94 530 27
94 526 28
99 575 29
100 578 30
102 580 31
106 610 32
107 615 33
107 609 34
108 604 35
112 648 36
112 648 37
114 660 38
114 654 39
114 644 40
114 644 41
115 653 42
115 649 43
116 660 44
120 694 45
122 704 46
122 700 47
122 712 48
123 715 49
123 715 50
123 715 51
125 723 52
125 721 53
126 708 54
126 706 55
125 673 56
126 682 57
127 689 58
127 677 59
127 671 60
127 669 61
128 674 62
129 681 63
129 681 64
130 688 65
129 673 66
131 691 67
132 698 68
132 694 69
133 699 70
133 699 71
134 718 72
134 716 73
134 716 74
135 727 75
135 723 76
136 728 77
136 728 78
136 728 79
138 744 80
139 749 81
140 764 82
141 773 83
142 796 84
142 778 85
143 783 86
144 790 87
145 793 88
145 777 89
146 792 90
146 792 91
147 797 92
147 795 93
147 795 94
149 

c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhou

23 109 1
33 157 2
39 195 3
40 190 4
50 254 5
55 277 6
57 295 7
61 319 8
64 336 9
68 366 10
70 374 11
71 367 12
71 363 13
73 371 14
76 392 15
79 415 16
79 423 17
80 428 18
82 434 19
85 457 20
86 460 21
88 484 22
89 493 23
91 503 24
91 503 25
93 525 26
93 525 27
97 565 28
98 558 29
98 558 30
98 558 31
99 567 32
100 570 33
100 570 34
101 569 35
101 561 36
104 582 37
105 583 38
108 594 39
108 594 40
109 599 41
110 602 42
110 596 43
111 579 44
112 582 45
112 576 46
112 574 47
113 579 48
113 577 49
115 593 50
116 598 51
117 605 52
117 601 53
119 635 54
119 635 55
121 657 56
122 662 57
123 661 58
125 677 59
125 677 60
126 678 61
127 683 62
127 683 63
128 692 64
129 699 65
129 691 66
131 703 67
131 689 68
131 687 69
131 681 70
131 675 71
131 675 72
131 671 73
132 680 74
132 680 75
133 689 76
133 683 77
133 681 78
133 675 79
133 675 80
133 675 81
134 684 82
135 695 83
135 691 84
135 691 85
136 696 86
136 696 87
136 696 88
135 683 89
135 683 90
136 682 91
136 680 92
137 691 93
137 683 94
138 686

c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhou

17 115 1
23 119 2
33 197 3
41 253 4
44 260 5
44 242 6
48 262 7
51 279 8
57 319 9
59 327 10
64 356 11
64 348 12
66 342 13
67 335 14
68 338 15
72 362 16
72 358 17
73 365 18
71 343 19
71 341 20
74 374 21
77 377 22
80 402 23
79 393 24
80 406 25
82 424 26
85 433 27
86 440 28
87 429 29
91 481 30
92 474 31
92 474 32
93 489 33
99 567 34
103 609 35
106 624 36
106 618 37
106 616 38
107 629 39
106 614 40
107 621 41
109 635 42
110 646 43
112 660 44
112 658 45
113 647 46
114 648 47
115 657 48
115 657 49
115 657 50
116 666 51
116 664 52
117 665 53
117 663 54
117 663 55
117 663 56
118 672 57
120 688 58
121 689 59
123 705 60
125 717 61
125 713 62
125 713 63
129 749 64
131 761 65
131 759 66
132 782 67
135 795 68
135 795 69
137 813 70
138 824 71
139 837 72
139 823 73
140 824 74
140 822 75
141 827 76
142 832 77
142 806 78
142 802 79
143 805 80
144 814 81
144 802 82
144 786 83
146 804 84
146 804 85
147 813 86
147 813 87
147 809 88
148 814 89
148 814 90
149 823 91
150 834 92
150 830 93
150 828 94
151 827 9

# Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:

with open('./benchmark_DC/'+dataset+'.csv','w') as f:
        
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        clf = DecisionTreeClassifier()
        clf.fit(X_train,y_train)

        num_trees = 1
        depth = clf.tree_.max_depth
        num_nodes = clf.tree_.node_count

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [9]:
import xgboost

In [45]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        xgb = xgboost.XGBClassifier(n_estimators=10,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        # xgb = xgboost.XGBClassifier()
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))
        train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier()
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        rfc = RandomForestClassifier()
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)